<a href="https://colab.research.google.com/github/adityathakekar/Agentic-AI/blob/main/Agentic_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ==============================================================================
# 🛠️ AUTOMATED PROJECT BUILDER: THE VC ANALYST SWARM
# ==============================================================================
import os
import shutil

# 1. SETUP PROJECT DIRECTORY
# ---------------------------
project_name = "VC_Analyst_Swarm"
dirs = [
    f"{project_name}/src",
    f"{project_name}/docs",
    f"{project_name}/tests"
]

# Clean up previous runs if they exist
if os.path.exists(project_name):
    shutil.rmtree(project_name)

# Create folders
for d in dirs:
    os.makedirs(d, exist_ok=True)

print(f"📂 Created Project Structure: {project_name}/")

# 2. WRITE THE CODE FILES
# ---------------------------

# --- FILE: src/config.py ---
config_code = """
import os
import google.generativeai as genai

def configure_api():
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("❌ GOOGLE_API_KEY not found in environment variables.")
    genai.configure(api_key=api_key)
    return api_key
"""
with open(f"{project_name}/src/config.py", "w") as f:
    f.write(config_code)

# --- FILE: src/tools.py ---
tools_code = """
from duckduckgo_search import DDGS

def search_web(query: str, max_results=3):
    \"\"\"
    Performs a real-time web search.
    Args:
        query (str): The topic to search for.
    \"\"\"
    print(f"   🔎 [TOOL CALL] Searching the web for: '{query}'...")
    try:
        results = DDGS().text(query, max_results=max_results)
        if not results:
            return "No results found."

        # Format the output for the AI
        summary = ""
        for r in results:
            summary += f"- TITLE: {r['title']}\\n  SNIPPET: {r['body']}\\n"
        return summary
    except Exception as e:
        return f"Search Error: {e}"
"""
with open(f"{project_name}/src/tools.py", "w") as f:
    f.write(tools_code)

# --- FILE: src/agents.py ---
agents_code = """
import google.generativeai as genai
from .tools import search_web

class Agent:
    def __init__(self, name, role, system_instruction):
        self.name = name
        self.role = role
        # We use Gemini 1.5 Flash (or Pro) as the brain
        self.model = genai.GenerativeModel(
            model_name='gemini-1.5-flash',
            system_instruction=system_instruction
        )

    def perform_task(self, task_description, context=""):
        prompt = f'''
        CONTEXT FROM PREVIOUS AGENTS:
        {context}

        YOUR CURRENT TASK:
        {task_description}

        INSTRUCTIONS:
        If you need to search, output exactly: [SEARCH: <query>]
        If you are done, output your final answer.
        '''

        # 1. First Pass: Think
        response = self.model.generate_content(prompt)
        text = response.text

        # 2. Tool Use Loop (Simple "ReAct" implementation)
        if "[SEARCH:" in text:
            # Extract query
            start = text.find("[SEARCH:") + 8
            end = text.find("]", start)
            query = text[start:end].strip()

            # Execute Tool
            search_result = search_web(query)

            # 3. Second Pass: Synthesize
            final_prompt = f"{prompt}\\n\\nSYSTEM INFO: Search Results for '{query}':\\n{search_result}\\n\\nNow provide the final answer based on this data."
            response = self.model.generate_content(final_prompt)
            return response.text

        return text

class SwarmManager:
    def __init__(self):
        # AGENT 1: The Researcher (Finds facts)
        self.researcher = Agent(
            "Researcher",
            "Data Gatherer",
            "You are a meticulous researcher. Your job is to find hard data, numbers, and recent news."
        )

        # AGENT 2: The Skeptic (Finds risks)
        self.skeptic = Agent(
            "Skeptic",
            "Risk Officer",
            "You are a cynical risk analyst. You look for downsides, competitors, and reasons why something might fail."
        )

        # AGENT 3: The Editor (Writes the report)
        self.editor = Agent(
            "Editor",
            "Publisher",
            "You are a professional editor. You synthesize conflicting reports into a beautiful Markdown document."
        )

    def run_mission(self, topic):
        print(f"🤖 [SWARM] Activating Agents for topic: {topic}")

        # Step 1
        print("\\n🕵️ [Researcher] Starting investigation...")
        data = self.researcher.perform_task(f"Find the top 3 startups and market size for {topic}.")

        # Step 2
        print("\\n⚖️ [Skeptic] Analyzing risks...")
        risks = self.skeptic.perform_task(f"What are the privacy risks and technical bottlenecks for {topic}?", context=data)

        # Step 3
        print("\\n📝 [Editor] Compiling final report...")
        final_report = self.editor.perform_task(
            "Create a structured Investment Memo. Include 'Market Opportunity' and 'Key Risks'.",
            context=f"RESEARCH DATA: {data}\\n\\nRISK DATA: {risks}"
        )

        return final_report
"""
with open(f"{project_name}/src/agents.py", "w") as f:
    f.write(agents_code)

# --- FILE: main.py ---
main_code = """
import os
import sys
from src.config import configure_api
from src.agents import SwarmManager

def main():
    # 1. Setup
    print("🚀 Initializing VC Analyst Swarm...")
    configure_api()

    # 2. Input
    topic = "Agentic AI Trends 2025"

    # 3. Execution
    swarm = SwarmManager()
    report = swarm.run_mission(topic)

    # 4. Output
    filename = "Final_Investment_Memo.md"
    with open(filename, "w") as f:
        f.write(report)

    print(f"\\n✅ SUCCESS! Report saved to {filename}")
    print("-" * 40)
    print(report)

if __name__ == "__main__":
    main()
"""
with open(f"{project_name}/main.py", "w") as f:
    f.write(main_code)

# --- FILE: README.md ---
readme_content = """
# 🕵️ VC Analyst Swarm

**An Agentic AI Architecture for Automated Due Diligence**

## 📖 Overview
VC Analyst Swarm is a multi-agent system designed to automate the initial phase of Venture Capital research. Instead of a single AI hallucinating facts, this system employs three distinct agents in a pipeline:

1. **The Researcher:** Scours the web for real-time market data.
2. **The Skeptic:** A specialized agent that "stress tests" the data to find risks.
3. **The Editor:** Synthesizes the conflict into a balanced Investment Memo.

## 🏗️ Architecture
The system uses a **Sequential Handoff Pattern**:
`[User Input] -> [Researcher Agent + Tool Use] -> [Skeptic Agent] -> [Editor Agent] -> [Markdown Report]`

Built with **Google Gemini 1.5 Flash** for low latency and high reasoning capabilities.

## 🚀 How to Run
1. Clone the repository.
2. Install dependencies: `pip install -r requirements.txt`
3. Set your API Key: `export GOOGLE_API_KEY=your_key`
4. Run: `python main.py`
"""
with open(f"{project_name}/README.md", "w") as f:
    f.write(readme_content)

# --- FILE: ARCHITECTURE.md ---
arch_content = """
# System Architecture

## Core Components

### 1. The Swarm Manager (`src/agents.py`)
Acts as the central orchestrator. It does not think; it directs traffic. It passes the output of one agent as the "Context" context to the next.

### 2. The Tool Layer (`src/tools.py`)
A modular wrapper around `DuckDuckGoSearch`. The agents are trained to emit a special token `[SEARCH: query]` which the system intercepts, executes, and feeds back into the prompt.

### 3. The Cognitive Layer (Gemini 1.5)
We utilize `gemini-1.5-flash` for its high tokens-per-second, allowing the swarm to complete a 3-step reasoning chain in under 15 seconds.
"""
with open(f"{project_name}/ARCHITECTURE.md", "w") as f:
    f.write(arch_content)

# --- FILE: requirements.txt ---
req_code = """
google-generativeai
duckduckgo-search
"""
with open(f"{project_name}/requirements.txt", "w") as f:
    f.write(req_code)

print("✅ Project Generation Complete!")
print(f"👉 Your code is now in the folder: {project_name}/")

📂 Created Project Structure: VC_Analyst_Swarm/
✅ Project Generation Complete!
👉 Your code is now in the folder: VC_Analyst_Swarm/


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [9]:
# ==============================================================================
# 🛠️ FIX: OVERWRITE src/agents.py WITH MODEL AUTO-DETECTION
# ==============================================================================
import os

project_name = "VC_Analyst_Swarm"

# New code for agents.py that finds a working model automatically
agents_code_fixed = """
import google.generativeai as genai
from .tools import search_web

# --- HELPER: FIND WORKING MODEL ---
def get_best_model_name():
    \"\"\"Scans available models to find one that works for this user.\"\"\"
    valid_models = []
    try:
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                valid_models.append(m.name)
    except Exception as e:
        # Fallback if list_models fails (rare)
        return 'gemini-pro'

    # Priority list (Newest to oldest)
    priorities = [
        "models/gemini-1.5-flash",
        "models/gemini-1.5-flash-001",
        "models/gemini-1.5-pro",
        "models/gemini-pro"
    ]

    for p in priorities:
        if p in valid_models:
            return p.replace("models/", "") # Strip prefix for initialization

    # Absolute fallback
    if valid_models:
        return valid_models[0].replace("models/", "")
    return "gemini-pro"

# Detect model once when module loads
ACTIVE_MODEL_NAME = get_best_model_name()
print(f"   [SYSTEM] Auto-selected Model: {ACTIVE_MODEL_NAME}")

class Agent:
    def __init__(self, name, role, system_instruction):
        self.name = name
        self.role = role
        # Use the auto-detected model
        self.model = genai.GenerativeModel(
            model_name=ACTIVE_MODEL_NAME,
            system_instruction=system_instruction
        )

    def perform_task(self, task_description, context=""):
        prompt = f'''
        CONTEXT FROM PREVIOUS AGENTS:
        {context}

        YOUR CURRENT TASK:
        {task_description}

        INSTRUCTIONS:
        If you need to search, output exactly: [SEARCH: <query>]
        If you are done, output your final answer.
        '''

        # 1. First Pass: Think
        try:
            response = self.model.generate_content(prompt)
            text = response.text
        except Exception as e:
            return f"Error generating content: {e}"

        # 2. Tool Use Loop (Simple "ReAct" implementation)
        if "[SEARCH:" in text:
            # Extract query
            try:
                start = text.find("[SEARCH:") + 8
                end = text.find("]", start)
                query = text[start:end].strip()

                # Execute Tool
                search_result = search_web(query)

                # 3. Second Pass: Synthesize
                final_prompt = f"{prompt}\\n\\nSYSTEM INFO: Search Results for '{query}':\\n{search_result}\\n\\nNow provide the final answer based on this data."
                response = self.model.generate_content(final_prompt)
                return response.text
            except Exception as e:
                return f"Error during tool use: {e}"

        return text

class SwarmManager:
    def __init__(self):
        # AGENT 1: The Researcher (Finds facts)
        self.researcher = Agent(
            "Researcher",
            "Data Gatherer",
            "You are a meticulous researcher. Your job is to find hard data, numbers, and recent news."
        )

        # AGENT 2: The Skeptic (Finds risks)
        self.skeptic = Agent(
            "Skeptic",
            "Risk Officer",
            "You are a cynical risk analyst. You look for downsides, competitors, and reasons why something might fail."
        )

        # AGENT 3: The Editor (Writes the report)
        self.editor = Agent(
            "Editor",
            "Publisher",
            "You are a professional editor. You synthesize conflicting reports into a beautiful Markdown document."
        )

    def run_mission(self, topic):
        print(f"🤖 [SWARM] Activating Agents for topic: {topic}")

        # Step 1
        print("\\n🕵️ [Researcher] Starting investigation...")
        data = self.researcher.perform_task(f"Find the top 3 startups and market size for {topic}.")

        # Step 2
        print("\\n⚖️ [Skeptic] Analyzing risks...")
        risks = self.skeptic.perform_task(f"What are the privacy risks and technical bottlenecks for {topic}?", context=data)

        # Step 3
        print("\\n📝 [Editor] Compiling final report...")
        final_report = self.editor.perform_task(
            "Create a structured Investment Memo. Include 'Market Opportunity' and 'Key Risks'.",
            context=f"RESEARCH DATA: {data}\\n\\nRISK DATA: {risks}"
        )

        return final_report
"""

# Overwrite the file
with open(f"{project_name}/src/agents.py", "w") as f:
    f.write(agents_code_fixed)

print("✅ FIX APPLIED: src/agents.py now auto-detects the correct model.")

✅ FIX APPLIED: src/agents.py now auto-detects the correct model.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [10]:
# ==============================================================================
# 🏆 VC ANALYST SWARM: FINAL & SILENT VERSION
# ==============================================================================
import os
import shutil
import sys
import warnings
import google.generativeai as genai
from getpass import getpass

# 1. SILENCE ALL WARNINGS (Fixes the infinite loop spam)
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

# 2. INSTALL DEPENDENCIES (Quietly)
print("⚙️ Installing libraries...")
!pip install -q -U google-generativeai duckduckgo-search

# 3. SETUP PROJECT FOLDER
project_name = "VC_Analyst_Swarm"
if os.path.exists(project_name):
    shutil.rmtree(project_name)
os.makedirs(f"{project_name}/src", exist_ok=True)

print(f"📂 Project initialized: {project_name}/")

# 4. GENERATE FILES

# --- src/tools.py ---
with open(f"{project_name}/src/tools.py", "w") as f:
    f.write("""
from duckduckgo_search import DDGS

def search_web(query: str, max_results=3):
    print(f"   🔎 [TOOL] Searching: '{query}'...")
    try:
        results = DDGS().text(query, max_results=max_results)
        if not results: return "No results found."
        return "\\n".join([f"- {r['title']}: {r['body']}" for r in results])
    except Exception as e:
        return f"Search Error: {e}"
""")

# --- src/agents.py ---
with open(f"{project_name}/src/agents.py", "w") as f:
    f.write("""
import google.generativeai as genai
from .tools import search_web

def get_working_model():
    # Priority list for models
    priority = ["models/gemini-1.5-flash", "models/gemini-1.5-pro", "models/gemini-pro"]

    # 1. Get available models from API
    try:
        my_models = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
    except:
        return "gemini-pro" # Safe fallback

    # 2. Match priority
    for p in priority:
        if p in my_models:
            print(f"   [SYSTEM] Using Model: {p}")
            return p.replace("models/", "")

    return "gemini-pro"

class SwarmManager:
    def __init__(self):
        self.model_name = get_working_model()

    def create_agent(self, role, instruction):
        return genai.GenerativeModel(
            model_name=self.model_name,
            system_instruction=f"ROLE: {role}. {instruction}"
        )

    def run_mission(self, topic):
        # Initialize Agents
        researcher = self.create_agent("Researcher", "Find facts. Output [SEARCH: query] if needed.")
        editor = self.create_agent("Editor", "Synthesize data into a Markdown report.")

        print(f"\\n🤖 [SWARM] Starting Analysis on: {topic}")

        # --- STEP 1: RESEARCH ---
        print(f"🕵️ [Researcher] Gathering intelligence...")
        prompt = f"Find top 3 startups and risks for: {topic}"

        # Generate first thought
        resp = researcher.generate_content(prompt)
        text = resp.text

        # Check for tool use
        if "[SEARCH:" in text:
            try:
                q = text.split("[SEARCH:")[1].split("]")[0].strip()
                data = search_web(q)
                # Feed data back to agent
                text = researcher.generate_content(f"{prompt}\\n\\nSEARCH DATA: {data}").text
            except:
                pass # Continue if search fails

        # --- STEP 2: EDITING ---
        print(f"📝 [Editor] Writing Investment Memo...")
        final = editor.generate_content(f"Write a comprehensive report based on this research:\\n{text}")
        return final.text
""")

# --- main.py ---
with open(f"{project_name}/main.py", "w") as f:
    f.write("""
import os
import google.generativeai as genai
from src.agents import SwarmManager

def main():
    # Load API Key
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key: return print("❌ No API Key found.")

    genai.configure(api_key=api_key)

    # Run Swarm
    try:
        swarm = SwarmManager()
        report = swarm.run_mission("Agentic AI Trends 2025")

        # Save output
        with open("Investment_Memo.md", "w") as f:
            f.write(report)
        print("\\n✅ MISSION SUCCESS! Saved to 'Investment_Memo.md'")
        print("-" * 40)
        print(report[:500] + "... (See file for full report)")

    except Exception as e:
        print(f"\\n❌ Error during execution: {e}")

if __name__ == "__main__":
    main()
""")

# --- README & ARCHITECTURE ---
with open(f"{project_name}/README.md", "w") as f:
    f.write("# VC Analyst Swarm\n\nAutomated Due Diligence Agents.")
with open(f"{project_name}/ARCHITECTURE.md", "w") as f:
    f.write("# Architecture\n\nSequential Multi-Agent System.")

print("✅ Files generated successfully.")

# 5. EXECUTE THE CODE
if "GOOGLE_API_KEY" not in os.environ:
    print("\n🔑 ENTER YOUR GOOGLE API KEY:")
    os.environ["GOOGLE_API_KEY"] = getpass()

# Add project to path and run
sys.path.append(os.path.abspath(project_name))
from VC_Analyst_Swarm.main import main

# Run main function
main()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

⚙️ Installing libraries...
📂 Project initialized: VC_Analyst_Swarm/
✅ Files generated successfully.
   [SYSTEM] Auto-detecting best available model...
   [SYSTEM] Selected Model: gemini-2.5-flash

🤖 [SWARM] Starting Analysis on: Agentic AI Trends 2025
🕵️ [Researcher] Gathering intelligence...
   🔎 [TOOL] Searching: 'define Agentic AI'...


/content/VC_Analyst_Swarm/src/tools.py:7: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for

📝 [Editor] Writing Investment Memo...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


✅ MISSION SUCCESS! Saved to 'Investment_Memo.md'
----------------------------------------
## Report: Agentic AI Trends 2025 - Top Startups and Key Risks

### Executive Summary

Agentic AI, defined as AI systems capable of autonomously planning, executing, and monitoring complex tasks, is poised for significant advancements by 2025. Key trends include enhanced reasoning, improved tool use, multi-agent collaboration, personalized automation, and broader industrial applications. This report identifies three leading startups — Adept AI, Imbue, and Cognition AI — that are at the forefront... (Check file for full report)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [11]:
import shutil
shutil.make_archive('VC_Analyst_Swarm', 'zip', 'VC_Analyst_Swarm')
print("✅ Zip created! Look for 'VC_Analyst_Swarm.zip' in the file browser on the left.")

✅ Zip created! Look for 'VC_Analyst_Swarm.zip' in the file browser on the left.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag